In [1]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np # to use numpy arrays instead of lists
import pandas as pd # DataFrame (table)
import matplotlib.pyplot as plt # to plot
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, Flatten, Dropout,  GlobalAveragePooling1D

from google.colab import drive
drive.mount('gdrive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
def leeDatos():

    dataSet = pd.read_csv("gdrive/MyDrive/Colab Notebooks/Amazon_Unlocked_Mobile.zip", header=0, compression='zip')

    return dataSet

In [ ]:
def encodeData(dataSet=0):
    dataSet.dropna(inplace=True)

    # Remove any 'neutral' ratings equal to 3
    dataSet = dataSet[dataSet['Rating'] != 3]

    # Encode 4s and 5s as 1 (rated positively)
    # Encode 1s and 2s as 0 (rated poorly)
    dataSet['Positively Rated'] = np.where(dataSet['Rating'] > 3, 1, 0)
    datos = ['Reviews', 'Positively Rated']
    misDatos = dataSet[datos]
    misDatos = misDatos.iloc[0:5000,:]

    return misDatos

In [ ]:
datos = leeDatos()
print(datos)
print("\n")

encodeDatos = encodeData(datos)
print(encodeDatos)
print("\n")
print(encodeDatos['Positively Rated'].mean())
print("\n")

NameError: name 'leeDatos' is not defined

In [ ]:
ts_size = .25
trainSet, testSet = train_test_split(encodeDatos, test_size=ts_size, shuffle=False, random_state=0)
print(trainSet)
print('trainSet shape: ', trainSet.shape)
print(testSet)
print('testSet shape: ', testSet.shape)
print("\n")

                                                Reviews  Positively Rated
0     I feel so LUCKY to have found this used (phone...                 1
1     nice phone, nice up grade from my pantach revu...                 1
2                                          Very pleased                 1
3     It works good but it goes slow sometimes but i...                 1
4     Great phone to replace my lost phone. The only...                 1
...                                                 ...               ...
5680  It is very good and almost as good as new. The...                 1
5681  Bought from UnlockedCellPhoneKings refurbished...                 1
5682  I bought this iphone as "new" and when i recei...                 0
5683  I'm not too interested in having the latest te...                 1
5684  Product was brand new, and was factory unlocke...                 1

[3750 rows x 2 columns]
trainSet shape:  (3750, 2)
                                                Reviews  Pos

In [ ]:
vocab_size = 10000
embedding_dim = 16
max_length = 100
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"

training_sentences= trainSet['Reviews']
training_labels = trainSet['Positively Rated']
testing_sentences = testSet['Reviews']
testing_labels = testSet['Positively Rated']

NameError: name 'trainSet' is not defined

In [ ]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)

word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [ ]:
training_padded = np.array(training_padded)
training_labels = np.array(training_labels)
testing_padded = np.array(testing_padded)
testing_labels = np.array(testing_labels)

In [ ]:
#dropout_rate = 0.25
model = Sequential([
                    Embedding(vocab_size, embedding_dim, input_length=max_length),
                    GlobalAveragePooling1D(),
                    Dense(24, activation='relu'),
                    #Dropout(dropout_rate),
                    #Dense(16, activation='relu'),
                    #Dropout(dropout_rate),
                    Dense(1, activation='sigmoid')
                    ])

opt = 'adam'
loss_fun = 'binary_crossentropy'
acc = ['accuracy']

model.compile(
    optimizer = opt,
    loss = loss_fun,
    metrics = acc
    )

NameError: name 'Sequential' is not defined

In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling1d             │ ?                           │     0 (unbuilt) │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
num_epochs = 30
history = model.fit(training_padded,
                    training_labels,
                    validation_split=0.25,
                    epochs=num_epochs,
                    verbose=2)

Epoch 1/30
88/88 - 7s - 75ms/step - accuracy: 0.7255 - loss: 0.5997 - val_accuracy: 0.6354 - val_loss: 0.6503
Epoch 2/30
88/88 - 1s - 11ms/step - accuracy: 0.7269 - loss: 0.5777 - val_accuracy: 0.6354 - val_loss: 0.6489
Epoch 3/30
88/88 - 1s - 16ms/step - accuracy: 0.7269 - loss: 0.5578 - val_accuracy: 0.6354 - val_loss: 0.6503
Epoch 4/30
88/88 - 1s - 7ms/step - accuracy: 0.7347 - loss: 0.5179 - val_accuracy: 0.6557 - val_loss: 0.5807
Epoch 5/30
88/88 - 1s - 7ms/step - accuracy: 0.7909 - loss: 0.4466 - val_accuracy: 0.6951 - val_loss: 0.5277
Epoch 6/30
88/88 - 1s - 7ms/step - accuracy: 0.8389 - loss: 0.3731 - val_accuracy: 0.7857 - val_loss: 0.4558
Epoch 7/30
88/88 - 1s - 7ms/step - accuracy: 0.8798 - loss: 0.3116 - val_accuracy: 0.7783 - val_loss: 0.4431
Epoch 8/30
88/88 - 0s - 4ms/step - accuracy: 0.9008 - loss: 0.2676 - val_accuracy: 0.8369 - val_loss: 0.3921
Epoch 9/30
88/88 - 0s - 4ms/step - accuracy: 0.9228 - loss: 0.2268 - val_accuracy: 0.7878 - val_loss: 0.4331
Epoch 10/30
88/8

In [ ]:
fig = plt.figure()
fig.add_subplot(121)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title("Loss vs Epochs")
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Training', 'Validation'], loc='upper right')
fig.add_subplot(122)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title("Accuracy vs Epochs")
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Training', 'Validation'], loc='upper right')
plt.show()

NameError: name 'plt' is not defined

In [ ]:
print(model.evaluate(training_padded, training_labels, verbose=2))
print("\n")

#### predict train data ######
df_train = pd.DataFrame()
df_train['pred'] = model.predict(training_padded).reshape(1,len(training_padded))[0]
df_train['real'] = training_labels
print(round(df_train))
print(model.evaluate(testing_padded,testing_labels, verbose=2))
print("\n")

# test data
df_test = pd.DataFrame()
df_test['pred'] = model.predict(testing_padded).reshape(1,len(testing_padded))[0]
df_test['real'] = testing_labels
print(round(df_test))


118/118 - 0s - 1ms/step - accuracy: 0.9531 - loss: 0.1581
[0.1580992341041565, 0.9530666470527649]


118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
      pred  real
0      1.0     1
1      1.0     1
2      1.0     1
3      1.0     1
4      1.0     1
...    ...   ...
3745   1.0     1
3746   1.0     1
3747   0.0     0
3748   1.0     1
3749   1.0     1

[3750 rows x 2 columns]
40/40 - 0s - 3ms/step - accuracy: 0.8768 - loss: 0.3249
[0.3248712122440338, 0.876800000667572]


40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
      pred  real
0      1.0     1
1      0.0     0
2      1.0     0
3      1.0     1
4      0.0     1
...    ...   ...
1245   1.0     0
1246   1.0     1
1247   0.0     0
1248   0.0     0
1249   1.0     1

[1250 rows x 2 columns]
